In [111]:
import csv
import math
import random 

lines = csv.reader(open('Pima.csv'))

dataset = list(lines)

for i in range(len(dataset)):
    if i == 0:
        continue
    dataset[i] = [float(x) for x in dataset[i]]

def splitData(dataset, splitRatio):
    trainSize = int(len(dataset)*splitRatio)
    trainSet = []
    copy = list(dataset)
    del copy[0]
    
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet,copy]

def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        if i == 0:
            continue
        vector = dataset[i]
        if vector[-1] not in separated:
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated


def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def mean(numbers):
    return sum(numbers)/ float(len(numbers))


def summarise(dataset):
    summaries = [(mean(att), stdev(att)) for att in zip(*dataset)]
    print('after summarise')
    print(summaries[-1])
    del summaries[-1]
    return summaries
    

def summariseByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue,instances in separated.items():
        summaries[classValue] = summarise(instances) 
    return summaries

def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi)*stdev))*exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
            return probabilities
    
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
        return bestLabel
    
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions
        

def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet)))*100.0

    
    
    
def main(dataSet):
    splitRatio = 0.67
    trainingSet, testSet = splitData(dataSet, splitRatio)
    
    print('Split {0} rows into train = {1} and test = {2} rows'.format(len(dataset),len(trainingSet),len(testSet)))
#     summarise the training data
    summaries = summariseByClass(trainingSet)
#     Test the data 
    predictions = getPredictions(summaries,testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))
 
main(dataset)


Split 769 rows into train = 515 and test = 253 rows
after summarise
(0.0, 0.0)
after summarise
(1.0, 0.0)
Accuracy: 61.26482213438735%
